In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [18]:
new = pd.read_excel('data_files/register_initial.xlsx', sheet_name='New Partner Register')
old = pd.read_excel('data_files/register_initial.xlsx', sheet_name='Old Partner Register')

In [19]:
new_updated = pd.merge(new, old, on='PARTNER')

In [23]:
conn = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"
df_customers = pd.read_sql_query(query, conn)
conn.close()

In [24]:
orders = pd.read_excel('data_files/temp.xlsx')

In [53]:
grouped = orders.groupby(["customer_name", "company_code_n", "sold_to_customer"])["order_intake_amount_eur"].sum().reset_index()

max_customer_number = grouped.groupby(["customer_name", "company_code_n"])["order_intake_amount_eur"].idxmax()
key_customers = grouped.loc[max_customer_number, ["customer_name", "company_code_n", "sold_to_customer"]]

other_customers = grouped[~grouped["sold_to_customer"].isin(key_customers["sold_to_customer"])]
other_customers = other_customers.groupby(["customer_name", "company_code_n"])["sold_to_customer"].apply(list).reset_index()

# Convert the 'sold_to_customer_y' column to integer in key_customers and other_customers DataFrames
key_customers['sold_to_customer'] = key_customers['sold_to_customer'].astype(int)
other_customers['sold_to_customer'] = other_customers['sold_to_customer'].apply(lambda x: ', '.join(map(str, map(int, x))))

final_df = key_customers.merge(other_customers, on=["customer_name", "company_code_n"], how="left")
final_df.rename(columns={"sold_to_customer": "key_id", "sold_to_customer_y": "list_of_other_ids"}, inplace=True)

writer = pd.ExcelWriter('data_files/outcome/check.xlsx')
# Save each DataFrame to a separate sheet in the same file
final_df.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()

In [70]:
grouped = orders.groupby(["customer_name", "tier_new", "type"]).first().reset_index()


# Create new columns for each unique value in "company_code_n":
unique_codes = orders["company_code_n"].unique()
for code in unique_codes:
    grouped[code] = ""

for index, row in grouped.iterrows():
    customer_name = row["customer_name"]
    tier_new = row["tier_new"]
    type_value = row["type"]
    for code in unique_codes:
        sales_people = orders.loc[(orders["customer_name"] == customer_name) & (orders["tier_new"] == tier_new) & (orders["type"] == type_value) & (orders["company_code_n"] == code) & (orders["FY"].dt.year.isin([2021, 2022, 2023])), "sales_person_n"].dropna().unique()
        sales_people = [str(x) for x in sales_people]  # Convert any float values to strings
        grouped.at[index, code] = ','.join(sales_people)
        
writer = pd.ExcelWriter('data_files/outcome/check_2.xlsx')
# Save each DataFrame to a separate sheet in the same file
grouped.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()

In [76]:
database1 = grouped[grouped['tier_new'].isin(['Channel Partner', 'Indirect Business Partner'])]

In [80]:
database1 = database1[['sold_to_customer', 'customer_name', 'type', 'company_code_n', 'tier_new', 'YEF-G', 'YEF-NL', 'YEF-PL', 'YEF-I', 'YEF-GB', 'YEF-CZ', 'YEF-B',
       'YEF-F', 'YEF-SK', 'YEF-HU', 'YEF-CH', 'YEF-E', 'YEF-RO', 'YEF-TR', 'YEF-A', 'YEF-HQ', 'YEF-RS']]

In [85]:
partner_register = pd.merge(new_updated, database1, left_on='PARTNER', right_on= 'customer_name', how='outer')

In [88]:
partner_register = pd.merge(partner_register, final_df, left_on='PARTNER', right_on= 'customer_name', how='outer')

In [90]:
writer = pd.ExcelWriter('data_files/outcome/partner_register.xlsx')
# Save each DataFrame to a separate sheet in the same file
partner_register.to_excel(writer, sheet_name='Partner register', index=False)
# Save the file
writer.close()

In [114]:
# understanding what products "for_bu" customers are buying

grouped = orders.groupby(["customer_name", "company_code_n", "for_bu"])["order_intake_amount_eur"].sum().reset_index()

max_customer_number = grouped.groupby(["customer_name", "company_code_n"])["order_intake_amount_eur"].idxmax()
key_customers = grouped.loc[max_customer_number, ["customer_name", "company_code_n", "for_bu"]]

other_customers = grouped.loc[~grouped.index.isin(max_customer_number)].copy()
other_customers = other_customers.groupby(["customer_name", "company_code_n"])["for_bu"].apply(lambda x: ','.join(map(str, x))).reset_index()

final_df = key_customers.merge(other_customers, on=["customer_name", "company_code_n"], how="left")
final_df.rename(columns={"for_bu": "key_bu", "sold_to_customer_y": "list_of_other_bus"}, inplace=True)

writer = pd.ExcelWriter('data_files/outcome/check_3.xlsx')
# Save each DataFrame to a separate sheet in the same file
final_df.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()